In [1]:
import torch
import torch.nn.functional as F
from torch.autograd import Variable
import os
import math
import data_loader
import resnet as models
import numpy as np
import numpy as np
import KMM_Lin

from sklearn.metrics import confusion_matrix
import torch.utils.data as Data

In [31]:
# def test(model, target_test_loader):
# #     model.eval() # 讓模型變成測試模式，主要針對Dropout與Batch Normalization在train與eval的不同設置模式
# #     test_loss = utils.AverageMeter() # Computes and stores the average and current value
#     correct_total = 0.
#     count_stack = 0
#     criterion = torch.nn.CrossEntropyLoss() # 定義一個標準準則，用來計算loss (讓輸出經過softmax，再進入Cross Entropy)
#     len_target_dataset = len(target_test_loader.dataset) #所有test資料集的總數
#     with torch.no_grad(): # 在做evaluation時，關閉計算導數來增加運行速度
#         for data, target in target_test_loader: # data為test資料，target為test label
#             data, target = data.to(DEVICE), target.to(DEVICE)
#             s_output = model.predict(data) # 將data放入模型得到預測的輸出
#             loss = criterion(s_output, target) #計算loss
# #             test_loss.update(loss.item()) # 更新值到紀錄中

#             # torch.max(a,1) 返回每一列中最大值的那個元素，且返回其索引
#             # troch.max()[1] 只返回最大值的每個索引
#             pred = torch.max(s_output, 1)[1]
#             correct_total += torch.sum(pred == target)

# #             pred_matrix = pred.data
# #             target_matrix = target.data
#             # print("pred_matrix、target_matrix")
#             # print(pred_matrix,target_matrix)
# #             if count_stack == 0:
# #                 pred_matrix_total = pred_matrix
# #                 target_matrix_total = target_matrix
# #                 count_stack =1
# #             elif count_stack == 1:
# #                 pred_matrix_total = torch.cat((pred_matrix_total,pred_matrix))
# #                 target_matrix_total = torch.cat((target_matrix_total,target_matrix))

#             # print("pred_matrix_total、target_matrix_total")
#             # print(pred_matrix_total, target_matrix_total)
# #         target_matrix_total = target_matrix_total.cpu().numpy()
# #         pred_matrix_total = pred_matrix_total.cpu().numpy()
# #         tn, fp, fn, tp = confusion_matrix(target_matrix_total, pred_matrix_total,labels=[0,1]).ravel()
#         # print(tn, fp, fn, tp)
#     # correct = correct.cpu().numpy()
#     # {:.2f}保留小數點後兩位
#     # print(correct)
#     # print(correct/300.)
#     # print('{:.3f}'.format(correct/len_target_dataset))
#     test_total = 100*correct_total.type(torch.float32)/len_target_dataset
#     print('test max correct: {}, test accuracy: {:.3f} % \n'.format(correct_total, test_total))

# #     logtest.append([test_total,tn, fp, fn, tp])
# #     np_log = np.array(logtest, dtype=float)
#     # delimiter : 分隔浮號 ； %.6f 浮點型保留六位小數
# #     np.savetxt(opt.save_test_name, np_log, delimiter=',', fmt='%.6f')


In [2]:
def test(model):
    model.eval()
    test_loss = 0
    correct = 0.
    correct1 = 0
    correct2 = 0
    correct3 = 0
    correct4 = 0
    count_stack = 0
    with torch.no_grad():
        for data, target in target_test_loader:
            if cuda:
                data, target = data.cuda(), target.cuda()
            data, target = Variable(data), Variable(target)
            pred1, pred2, pred3, pred4 = model(data)

            pred1 = torch.nn.functional.softmax(pred1, dim=1)
            pred2 = torch.nn.functional.softmax(pred2, dim=1)
            pred3 = torch.nn.functional.softmax(pred3, dim=1)
            pred4 = torch.nn.functional.softmax(pred4, dim=1)

            pred = (pred1 + pred2 + pred3 + pred4) / 4
            test_loss += F.nll_loss(F.log_softmax(pred, dim=1), target).item()  # sum up batch loss

            pred = pred.data.max(1)[1]  # get the index of the max log-probability
            correct += pred.eq(target.data.view_as(pred)).cpu().sum()
            
            pred = pred1.data.max(1)[1]  # get the index of the max log-probability
            correct1 += pred.eq(target.data.view_as(pred)).cpu().sum()
            
            pred = pred2.data.max(1)[1]  # get the index of the max log-probability
            correct2 += pred.eq(target.data.view_as(pred)).cpu().sum()
            
            pred = pred3.data.max(1)[1]  # get the index of the max log-probability
            correct3 += pred.eq(target.data.view_as(pred)).cpu().sum()
            
            pred = pred4.data.max(1)[1]  # get the index of the max log-probability
            correct4 += pred.eq(target.data.view_as(pred)).cpu().sum()

#             pred_matrix = pred.data
#             target_matrix = target.data
#             if count_stack == 0:
#                 pred_matrix_total = pred_matrix
#                 target_matrix_total = target_matrix
#                 count_stack =1
#             elif count_stack == 1:
#                 pred_matrix_total = torch.cat((pred_matrix_total,pred_matrix))
#                 target_matrix_total = torch.cat((target_matrix_total,target_matrix))

#         target_matrix_total = target_matrix_total.cpu().numpy()
#         pred_matrix_total = pred_matrix_total.cpu().numpy()
#         tn, fp, fn, tp = confusion_matrix(target_matrix_total, pred_matrix_total,labels=[0,1]).ravel()

        test_loss /= len(target_test_loader.dataset)
        print(target_test_name, '\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
            test_loss, correct.type(torch.float32), len(target_test_loader.dataset),100. * correct.type(torch.float32) / len(target_test_loader.dataset)))
        
        print('\nsource1 accnum {}, source2 accnum {}，source3 accnum {}，source4 accnum {}'.format(
            correct1.type(torch.float32), correct2.type(torch.float32), correct3.type(torch.float32), correct4.type(torch.float32)))

        test_total = 100*correct.type(torch.float32)/len(target_test_loader.dataset)

#         logtest.append([test_total,tn, fp, fn, tp,correct1,correct2,correct3,correct4])
#         np_log = np.array(logtest, dtype=float)
#         np.savetxt(opt.save_test_name, np_log, delimiter=',', fmt='%.6f')

    return correct.type(torch.float32)

In [23]:
# def load_data(root_dir):
#     batch_size = 8
#     folder_test = root_dir
#     target_test_loader,test_index = data_loader.load_data(folder_test, batch_size, False, CFG['kwargs'])
#     return target_test_loader

In [4]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
cuda = True

kwargs = {'num_workers': 4, 'pin_memory': True} if cuda else {}
root_path = "E:\\AmigoChou\\Q2\Training\\M2O_datasets\\instance\\4_to_1\\_to_pear\\59"
target_test_name = "\\target\\test"
batch_size = 8

target_test_loader = data_loader.load_testing(root_path, target_test_name, batch_size, kwargs)

In [8]:
# DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = models.MFSAN(num_classes=2)
model.load_state_dict(torch.load('E:\\AmigoChou\\Q2\\Training\\M2O_LXZ_editbyAmigo\\4_to_1\\log_model\\_to_pear\\59_1\\model_mmd_parameter_1_log0.pkl'))
model.cuda()
model.eval()
# model = torch.load('model_parameter_1_log1.pkl')

MFSAN(
  (sharedNet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats

In [9]:
# target_test_loader = load_data('E:\\AmigoChou\\Q2\\Training\\O2M_LXZ vs ZYM\\datasets\\O2M_datasets_backup\\apple_to_\\to_pear\\sn_200_sp_200\\target\\test')
test(model)

\target\test 
Test set: Average loss: 0.0747, Accuracy: 502.0/600 (83.67%)


source1 accnum 470.0, source2 accnum 470.0，source3 accnum 435.0，source4 accnum 538.0


tensor(502.)